In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata


In [2]:
idNotWork = 292590
idWork=219598

In [ ]:
data = loadOSCdata.queryOSCapi(OSCid = idWork, output = 'csv', outputFile = 'data.csv',dataType='photos')
data.head()